## TASK 2: Quantum Generative Adversarial Network (QGAN) 

Dataset: QIS_EXAM_200Events.npz

- This notebook consists of working implementation of QGAN for the given dataset and at the end tensorboard profiling is shown
- Used code snippets from tutorials of tensorflow Quantum
- Skip the following block, if tensorflow and tensorflow quantum are installed already.




In [ ]:
!pip install tensorflow==2.3.1
!pip install tensorflow_quantum

In [ ]:

import tensorflow as tf
import tensorflow_quantum as tfq
from tensorflow.keras import layers
import cirq
import sympy
import numpy as np

%load_ext tensorboard

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

### Loading Data

Get training data, labels and test data and corresponding labels

The .npz file consists of zero-dimensional array, accessing which requires expanding its dimensions along axis=0

In [ ]:
data = np.load("/content/QIS_EXAM_200Events.npz", allow_pickle=True)

for k in data:
    print(k)
total_data = dict(zip(("train_data","test_data"), (data[k] for k in data)))
total_data = np.expand_dims(total_data,axis=0)

train_data = total_data[0]["train_data"]
test_data = total_data[0]["test_data"]
training_data_signal = np.expand_dims(train_data,axis=0)[0]['1']
training_data_background = np.expand_dims(train_data,axis=0)[0]['0']



In [ ]:
def generate_circuit(qubits):
    """Generate a random circuit on qubits."""
    random_circuit = cirq.generate_boixo_2018_supremacy_circuits_v2(
        qubits, cz_depth=2, seed=1234)
    return random_circuit

In [ ]:
N_QUBITS = 5
QUBITS = cirq.GridQubit.rect(1, N_QUBITS)
REFERENCE_CIRCUIT = generate_circuit(QUBITS)
all_data = training_data_signal + training_data_background

Creating Generator and discriminator models

In [ ]:
LATENT_DIM = 100
def make_generator_model():
    """Construct generator model."""
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(LATENT_DIM,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(N_QUBITS, activation='relu'))

    return model

def make_discriminator_model():
    """Constrcut discriminator model."""
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(N_QUBITS,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1))

    return model


In [ ]:
discriminator = make_discriminator_model()
generator = make_generator_model()


In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output, fake_output):
    """Compute discriminator loss."""
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    """Compute generator loss."""
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [ ]:
BATCH_SIZE=256

@tf.function
def train_step(images):
    """Run train step on provided image batch."""
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        # print(images.shape)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(
        gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(
        disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(
        zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(
        zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss


In [ ]:
import datetime
logdir = "tb_logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()


The Generator loss and Discriminator loss can be vizualized using tensorboard.

In [ ]:
def train(dataset, epochs, start_epoch=1):
    """Launch full training run for the given number of epochs."""
    # Log original training distribution.
    # tf.summary.histogram('Training Distribution', data=bits_to_ints(dataset), step=0)

    batched_data = tf.data.Dataset.from_tensor_slices(dataset).shuffle(100).batch(512)
    t = time.time()
    for epoch in range(start_epoch, start_epoch + epochs):
        for i, image_batch in enumerate(batched_data):
            # Log batch-wise loss.
            gl, dl = train_step(image_batch)
            tf.summary.scalar(
                'Generator loss', data=gl, step=epoch * len(batched_data) + i)
            tf.summary.scalar(
                'Discriminator loss', data=dl, step=epoch * len(batched_data) + i)
        if epoch % 10 == 0:
            print('Epoch {}, took {}(s)'.format(epoch, time.time() - t))
            t = time.time()


In [ ]:
%tensorboard --logdir tb_logs/

logdir = "tb_logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()


Profiling can also be visualized using tensorboard

In [ ]:
import time
tf.profiler.experimental.start(logdir)
train(all_data, epochs=100, start_epoch=10)
tf.profiler.experimental.stop()


In [ ]:
test_data_signal = np.expand_dims(test_data,axis=0)[0]['1']

In [ ]:
val_signal = discriminator(test_data_signal)

In [ ]:
print(f"accuracy: " + str(sum([1 if i>0 else 0 for i in val_signal])/len(val_signal)))
